This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv("nasdaq.env")
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key=' + API_KEY
r = requests.get(url)

In [4]:
r

<Response [200]>

In [5]:
json_data = r.json()

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [7]:
r = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)
r

<Response [200]>

In [8]:
json_data = r.json()

In [9]:
# Going 1 step down into dataset dict
data = json_data['dataset_data']

In [10]:
data['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [11]:
data['data'][0]

['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

In [12]:
data['data'][0][1]

51.76

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [13]:
opening_prices = []
for daily in data['data']:
    if daily[1] != None:
        opening_prices.append(daily[1])

In [14]:
opening_prices.sort()

In [15]:
print('The highest opening price was: $',opening_prices[-1])
print('The lowest opening price was: $',opening_prices[0])

The highest opening price was: $ 53.11
The lowest opening price was: $ 34.0


#### 4. What was the largest change in any one day (based on High and Low price)?

In [16]:
gap_prices = []
for daily in data['data']:
    gap_prices.append(daily[2] - daily[3])

In [17]:
print('The largest change in any one day was $',round(max(gap_prices),2))

The largest change in any one day was $ 2.81


#### 5. What was the largest change between any two days (based on Closing Price)?

In [18]:
closing_prices = []
for daily in data['data']:
    closing_prices.append(daily[4])

In [19]:
gap_closing=[]
counter = 0

while counter < len(closing_prices)-1:
    gap_closing.append(closing_prices[counter+1]-closing_prices[counter])
    counter += 1

In [20]:
print('The largest change between any two days was $',round(max(gap_closing),2))

The largest change between any two days was $ 2.56


#### 6. What was the average daily trading volume during this year?

In [21]:
traded_volume = []
for daily in data['data']:
    traded_volume.append(daily[6])

In [22]:
print('The average daily trading was $',round(sum(traded_volume)/len(traded_volume),2))

The average daily trading was $ 89124.34





#### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [23]:
traded_volume.sort()

In [24]:
if len(traded_volume) % 2 == 0:
    median = (((len(traded_volume)/2)-1) + (len(traded_volume)/2)) / 2
else:
    median = (len(traded_volume) - 1) / 2

In [26]:
print('The median trading volume this year was: $', traded_volume[int(median)])

The median trading volume this year was: $ 76286.0
